In [12]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)

### Read in Solar Power Data

In [13]:
full_data = pd.read_csv('../Data/GAC PV & Battery Data 11-20-2020.csv', low_memory=False)
full_data.Date = pd.to_datetime(full_data.Date, format='%d-%b-%y %H:%M:%S')
full_data = full_data.loc[full_data.Date.dt.date > pd.to_datetime('2020-02-29')]
full_data = full_data.loc[~full_data['GAC PV Electric KW'].isin(['I/O Timeout', 'No Result'])].copy()
full_data = full_data[['Date', 'GAC PV Electric KW']]
full_data[['GAC PV Electric KW']] = full_data[['GAC PV Electric KW']].astype(float)
full_data.head()

,Date,GAC PV Electric KW
41760,2020-03-01 00:00:00,-0.029025
41761,2020-03-01 00:01:00,-0.028887
41762,2020-03-01 00:02:00,-0.028749
41763,2020-03-01 00:03:00,-0.028610
41764,2020-03-01 00:04:00,-0.028472


### Solar Radiation Data

#### Required functions for calculating solar radiation and solar constants

In [14]:
def eot(date):
    d = ((date - pd.to_datetime('2020-01-01')).dt.days) + 1
    B = (360/365) * (d-81)  * (np.pi/180)
    return 9.87 * np.sin(2*B) - 7.53*np.cos(B) - 1.5*np.sin(B)
def declination(date):
    d = ((date - pd.to_datetime('2020-01-01')).dt.days) + 1
    return -23.45 * np.cos(360/365 * (d + 10) * (np.pi / 180))
def dist_to_earth_actual(date):
    d = ((date - pd.to_datetime('2020-01-03')).dt.days)
    a = 149600000e3
    e = 0.0167
    theta = 2*np.pi*d / 365.256
    return a * (1-e**2) / (1+e*np.cos(theta))
def azimuth(x):
    # x = [hour_angle, zenith, solar_declination]
    a = np.arccos((np.sin(43.1566 * (np.pi/180)) * np.cos(x[1] * (np.pi/180)) - np.sin(x[2] * (np.pi/180))) / (np.cos(43.1566 * (np.pi/180)) * np.sin(x[1] * (np.pi/180)))) * (180/np.pi)
    if x[0] > 0:
        return (a + 180) % 360
    else:
        return (540-a) % 360

#### Use the times from the solar panel data to calculate the solar flux/radiation

In [15]:
tilt_angle = 0 # Degrees
zenith_data = full_data[['Date', 'GAC PV Electric KW']].copy()
zenith_data['EOT'] = zenith_data[['Date']].apply(eot).copy()
zenith_data['Local_time_mins'] = zenith_data.Date.dt.hour * 60 + zenith_data.Date.dt.minute
zenith_data['true_solar_time'] = (zenith_data['Local_time_mins'] + zenith_data['EOT'] + 4*-77.6088 - 60*-5) % 1440
zenith_data['Hour_Angle'] = zenith_data['true_solar_time'].apply(lambda x: x/4 + 180 if x/4 < 0 else x/4-180)
zenith_data['solar_declination'] = zenith_data[['Date']].apply(declination)
zenith_data['cos_zenith'] = (np.sin(43.1566 * (np.pi/180)) * np.sin(zenith_data['solar_declination'] * (np.pi / 180)) + 
                             np.cos(43.1566 * (np.pi / 180)) * np.cos(zenith_data['solar_declination'] * (np.pi / 180)) * np.cos(zenith_data['Hour_Angle'] * (np.pi / 180)))

zenith_data['cos_incidence'] = (np.sin((43.1566 - tilt_angle) * (np.pi/180)) * np.sin(zenith_data['solar_declination'] * (np.pi / 180)) + 
                               np.cos((43.1566 - tilt_angle) * (np.pi / 180)) * np.cos(zenith_data['solar_declination'] * (np.pi / 180)) * np.cos(zenith_data['Hour_Angle'] * (np.pi / 180)))
zenith_data['actual_distance'] = zenith_data[['Date']].apply(dist_to_earth_actual)
zenith_data['solar_flux'] = 1.37 *  (149600000e3 / zenith_data['actual_distance'])**2 * zenith_data['cos_zenith'] # Solar flux in kilowatts
zenith_data['solar_flux_corrected'] = 1.37 *  (149600000e3 / zenith_data['actual_distance'])**2 * zenith_data['cos_incidence']
zenith_data['zenith'] = np.arccos(zenith_data['cos_zenith']) * (180/np.pi)
zenith_data['azimuth'] = zenith_data[['Hour_Angle', 'zenith', 'solar_declination']].apply(azimuth, axis=1)
zenith_data.head()

,Date,GAC PV Electric KW,EOT,Local_time_mins,true_solar_time,Hour_Angle,solar_declination,cos_zenith,cos_incidence,actual_distance,solar_flux,solar_flux_corrected,zenith,azimuth
41760,2020-03-01 00:00:00,-0.029025,-12.853552,0,1416.711248,174.177812,-8.009835,-0.813954,-0.813954,1.482162e+11,-1.136036,-1.136036,144.484044,350.042607
41761,2020-03-01 00:01:00,-0.028887,-12.853552,1,1417.711248,174.427812,-8.009835,-0.814267,-0.814267,1.482162e+11,-1.136473,-1.136473,144.514916,350.465587
41762,2020-03-01 00:02:00,-0.028749,-12.853552,2,1418.711248,174.677812,-8.009835,-0.814566,-0.814566,1.482162e+11,-1.136891,-1.136891,144.544459,350.889162
41763,2020-03-01 00:03:00,-0.028610,-12.853552,3,1419.711248,174.927812,-8.009835,-0.814851,-0.814851,1.482162e+11,-1.137289,-1.137289,144.572670,351.313306
41764,2020-03-01 00:04:00,-0.028472,-12.853552,4,1420.711248,175.177812,-8.009835,-0.815123,-0.815123,1.482162e+11,-1.137669,-1.137669,144.599546,351.737995


### Weather Data

In [33]:
w_data = pd.read_csv('..\Data\Weather_ROC_2020.csv', low_memory=False)
w_data = w_data.drop('STATION', axis=1) # There is only one station recorded
w_data.DATE = pd.to_datetime(w_data.DATE, format='%Y-%m-%dT%H:%M:%S')
w_data = w_data.loc[(w_data.REPORT_TYPE.astype(str) == 'FM-15'), ['DATE', 'HourlySkyConditions', 'HourlyDryBulbTemperature', 'HourlyRelativeHumidity']].copy()
w_data = w_data.loc[~(w_data.HourlyDryBulbTemperature == '*')].copy()
for i in np.where(w_data.isnull().HourlySkyConditions.values)[0]:
    w_data.HourlySkyConditions.iloc[i] = w_data.HourlySkyConditions.iloc[i-1]
w_data.isnull().sum(axis=0)
w_data['NumericCloud'] = w_data['HourlySkyConditions'].astype(str).apply(lambda x: x.split(':')[-1][0:2]).astype(float)
w_data['HourlyDryBulbTemperature'] = w_data['HourlyDryBulbTemperature'].astype(float)
w_data['HourlyRelativeHumidity'] = w_data['HourlyRelativeHumidity'].astype(float)
w_data.head()

,DATE,HourlySkyConditions,HourlyDryBulbTemperature,HourlyRelativeHumidity,NumericCloud
0,2020-01-01 00:54:00,BKN:07 50 BKN:07 70,30.0,69.0,7.0
2,2020-01-01 01:54:00,FEW:02 55 BKN:07 75,30.0,69.0,7.0
4,2020-01-01 02:54:00,BKN:07 46,30.0,69.0,7.0
5,2020-01-01 03:54:00,OVC:08 45,30.0,66.0,8.0
6,2020-01-01 04:54:00,SCT:04 44 OVC:08 60,30.0,66.0,8.0


### Join the two data frames

In [34]:
zenith_data_hourly = zenith_data.groupby([zenith_data.Date.dt.date, zenith_data.Date.dt.hour]).agg(
    mean_power = ('GAC PV Electric KW', np.mean),
    mean_radiation = ('solar_flux', np.mean))
zenith_data_hourly.index.names = ['Date', 'Hour']
zenith_data_hourly.reset_index(inplace=True)

In [36]:
w_data_hourly = w_data.groupby([w_data.DATE.dt.date, w_data.DATE.dt.hour]).agg(
    mean_temp = ('HourlyDryBulbTemperature', np.mean),
    mean_cloud = ('NumericCloud', np.mean),
    mean_humid = ('HourlyRelativeHumidity', np.mean))
w_data_hourly.index.names = ['Date', 'Hour']
w_data_hourly.reset_index(inplace=True)

In [47]:
merged_data = w_data_hourly.merge(zenith_data_hourly)
merged_data = merged_data.loc[(merged_data.Hour >= 8) & (merged_data.Hour <= 16)] # Times between 8am and 4pm
daily_data = merged_data.groupby('Date', as_index=False).agg(
    mean_power = ('mean_power', np.mean),
    mean_radiation = ('mean_radiation', np.mean),
    mean_temp = ('mean_temp', np.mean),
    mean_humid = ('mean_humid', np.mean),
    mean_cloud = ('mean_cloud', np.mean))
daily_data['mean_panel'] = daily_data['mean_power'] / 1570.061376 # This is the flux of the panels, we add this to have a comparison metric to solar radiation
# 1570.061376 is a rough estimate for the surface area of the panels

In [48]:
daily_data.to_csv('daily_aggregate_data.csv', index=False)